In [2]:
print ( 'hello world')

hello world


In [3]:
import pandas as pd
import numpy as np
df = pd.read_csv('data/ad_feature.csv',header=0)

In [4]:
df.head()

,adgroup_id,cate_id,campaign_id,customer,brand,price
0,63133,6406,83237,1,95471.0,170.00
1,313401,6406,83237,1,87331.0,199.00
2,248909,392,83237,1,32233.0,38.00
3,208458,392,83237,1,174374.0,139.00
4,110847,7211,135256,2,145952.0,32.99


In [5]:
np.linspace(0,1,11)

array([0. , 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1. ])

In [6]:
c,  bins = pd.qcut( df['price'], q = np.linspace(0,1,11) ,retbins=True,  duplicates='drop')  
price_level, ind  = pd.factorize(c, sort=True)
df['price_level'] = price_level 


In [7]:
df.head()

,adgroup_id,cate_id,campaign_id,customer,brand,price,price_level
0,63133,6406,83237,1,95471.0,170.00,5
1,313401,6406,83237,1,87331.0,199.00,6
2,248909,392,83237,1,32233.0,38.00,1
3,208458,392,83237,1,174374.0,139.00,4
4,110847,7211,135256,2,145952.0,32.99,1


In [9]:
## 离群点处理 
des = df['price'].describe() 
print(des)


count    8.468110e+05
mean     1.838867e+03
std      3.108877e+05
min      1.000000e-02
25%      4.900000e+01
50%      1.390000e+02
75%      3.520000e+02
max      1.000000e+08
Name: price, dtype: float64


In [10]:
valid_max =  des['50%'] + 3* ( des['75%'] -des['50%'] ) 

df['price'] = df['price'].clip(0, valid_max)   

#观察下输出的最大值，已经把异常值给clip了
print( df['price'].describe())


count    846811.000000
mean        243.996387
std         253.800315
min           0.010000
25%          49.000000
50%         139.000000
75%         352.000000
max         778.000000
Name: price, dtype: float64


In [11]:

####### 接下来统计下 各个品牌的平均价格 
g = df.groupby('brand')
m = g['price'].mean() 

#只看下前10行
m[:10] 


brand
2.0      53.412500
3.0     739.368421
7.0     217.636364
8.0      32.450000
9.0      44.200000
10.0    158.916667
Name: price, dtype: float64

In [12]:
## 这里用另外一个简单数据来介绍下 axis 概念，很多复杂操作依赖于对这个的理解  
a = { 'A' : [ 1, 2, 3 ] , 'B' : [100,200, 300 ]} 
d = pd.DataFrame( a ) 
print(d)


   A    B
0  1  100
1  2  200
2  3  300


In [13]:
d.sum(axis=0 )  #axis = 0表示 按照行遍历进行sum  

A      6
B    600
dtype: int64

In [14]:
d.sum(axis=1)  # axis=1 表示 按照列遍历 进行sum  

0    101
1    202
2    303
dtype: int64

In [15]:
# 再来看一下d ，参照一下
d

,A,B
0,1,100
1,2,200
2,3,300


In [16]:
d['sum'] =  d.sum(axis=1)
d

,A,B,sum
0,1,100,101
1,2,200,202
2,3,300,303


In [17]:
f =  d.sum(axis=0)  
f

A        6
B      600
sum    606
dtype: int64

In [23]:
d=  d.append(f,ignore_index=True)
print(d)

   A    B  sum
0  1  100  101
1  2  200  202
2  3  300  303
3  6  600  606


In [24]:
d.index = [ 0 , 1, 2 , 'sum'] 
print(d)

     A    B  sum
0    1  100  101
1    2  200  202
2    3  300  303
sum  6  600  606


In [2]:
##### concat, merge  等功能
import pandas as pd
df1 = pd.DataFrame({'A': ['A0', 'A1', 'A2', 'A3'],
                        'B': ['B0', 'B1', 'B2', 'B3'],
                        'C': ['C0', 'C1', 'C2', 'C3'],
                        'D': ['D0', 'D1', 'D2', 'D3']},
                    index=[0, 1, 2, 3])
df2 = pd.DataFrame({'A': ['A4', 'A5', 'A6', 'A7'],
                         'B': ['B4', 'B5', 'B6', 'B7'],
                        'C': ['C4', 'C5', 'C6', 'C7'],
                       'D': ['D4', 'D5', 'D6', 'D7']},
                         index=[4, 5, 6, 7])

In [4]:
print(df1,'\n\n' , df2)

    A   B   C   D
0  A0  B0  C0  D0
1  A1  B1  C1  D1
2  A2  B2  C2  D2
3  A3  B3  C3  D3 

     A   B   C   D
4  A4  B4  C4  D4
5  A5  B5  C5  D5
6  A6  B6  C6  D6
7  A7  B7  C7  D7


In [5]:
pd.concat([df1,df2],axis=0) #axis=0，按照行来合并 

,A,B,C,D
0,A0,B0,C0,D0
1,A1,B1,C1,D1
2,A2,B2,C2,D2
3,A3,B3,C3,D3
4,A4,B4,C4,D4
5,A5,B5,C5,D5
6,A6,B6,C6,D6
7,A7,B7,C7,D7


In [6]:
pd.concat([df1,df2],axis=1) # axis=1 ,按照列来合并 ，这里注意到 index 对concat的影响 

,A,B,C,D,A,B,C,D
0,A0,B0,C0,D0,NaN,NaN,NaN,NaN
1,A1,B1,C1,D1,NaN,NaN,NaN,NaN
2,A2,B2,C2,D2,NaN,NaN,NaN,NaN
3,A3,B3,C3,D3,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,A4,B4,C4,D4
5,NaN,NaN,NaN,NaN,A5,B5,C5,D5
6,NaN,NaN,NaN,NaN,A6,B6,C6,D6
7,NaN,NaN,NaN,NaN,A7,B7,C7,D7


In [7]:
pd.concat([df1, df2.reset_index(drop=True)], axis=1 ) #把df2的 index重置以后，再进行列合并 


,A,B,C,D,A,B,C,D
0,A0,B0,C0,D0,A4,B4,C4,D4
1,A1,B1,C1,D1,A5,B5,C5,D5
2,A2,B2,C2,D2,A6,B6,C6,D6
3,A3,B3,C3,D3,A7,B7,C7,D7


In [8]:
####  merge - how = {inner, left, right ,outer }  inner ,left 比较常用 
left = pd.DataFrame({'key': ['K0', 'K1', 'K2', 'K3'],
                    'A': ['A0', 'A1', 'A2', 'A3'],
                         'B': ['B0', 'B1', 'B2', 'B3']})
right = pd.DataFrame({'key': ['K0', 'K1', 'K2', 'K4'],
                          'C': ['C0', 'C1', 'C2', 'C3'],
                         'D': ['D0', 'D1', 'D2', 'D3']}) 

print(left,'\n\n',right)

    A   B key
0  A0  B0  K0
1  A1  B1  K1
2  A2  B2  K2
3  A3  B3  K3 

     C   D key
0  C0  D0  K0
1  C1  D1  K1
2  C2  D2  K2
3  C3  D3  K4


In [9]:
pd.merge(left, right , on='key',how='inner') #on='key' 表示按照 key 这一列来进行merge 相等key的行merge到一起    

,A,B,key,C,D
0,A0,B0,K0,C0,D0
1,A1,B1,K1,C1,D1
2,A2,B2,K2,C2,D2


In [10]:
pd.merge(left, right , on='key',how='left') # left merge , left的数据全部保留，merge不上的，补null值  

,A,B,key,C,D
0,A0,B0,K0,C0,D0
1,A1,B1,K1,C1,D1
2,A2,B2,K2,C2,D2
3,A3,B3,K3,NaN,NaN


In [11]:
#大家自己试试 right，outer的情况吧 :-)  
print ("Let's try it ")

Let's try it 


In [13]:
### 最灵活的方法apply ，先进行入门: 
df = pd.read_csv('data/ad_feature.csv',header=0) 
df.head()


,adgroup_id,cate_id,campaign_id,customer,brand,price
0,63133,6406,83237,1,95471.0,170.00
1,313401,6406,83237,1,87331.0,199.00
2,248909,392,83237,1,32233.0,38.00
3,208458,392,83237,1,174374.0,139.00
4,110847,7211,135256,2,145952.0,32.99


In [14]:
##使用apply，只把price这一列取出来  NOTE：这里是为了演示apply的功能，先理解了简单用法，后续复杂功能才能理解

df[:10].apply( lambda x: x.price, axis=1) # axis=1非常关键， 它是说： 每次执行遍历列，也就是把 一整行送进去  

0    170.00
1    199.00
2     38.00
3    139.00
4     32.99
5    199.00
6     99.00
7     33.00
8     19.00
9    428.00
dtype: float64

In [15]:
df[:10].apply(lambda x : x.max(), axis=0) #  axis =0,每次执行遍历行，也就是把一整列 送进去，那么这个语句的输出含义 是什么呢? 

adgroup_id     607788.0
cate_id          7213.0
campaign_id    395195.0
customer           13.0
brand          207800.0
price             428.0
dtype: float64

In [16]:
print("Try it yourself..Make a Fun ")

Try it yourself..Make a Fun 
